<a href="https://colab.research.google.com/github/Lodia15/Monet_Paintings_Lodia/blob/main/next_Monet_Lodia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
! pip install kaggle

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
!mkdir -p ~/.kaggle

In [31]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [32]:
! chmod 600 ~/.kaggle/kaggle.json

In [33]:
!kaggle datasets list


ref                                                                title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
wardabilal/spotify-global-music-dataset-20092025                   Spotify Global Music Dataset (2009–2025)               1289021  2025-11-11 09:43:05.933000          13401        301  1.0              
ranaghulamnabi/shopping-behavior-and-preferences-study             Shopping Behavior & Preferences Study                    72157  2025-12-03 09:14:26.797000           1317         28  1.0              
rohiteng/amazon-sales-dataset                                      Amazon Sales Dataset                                   4037578  2025-11-23 14:29:37.973000           4832         67  1.0

In [34]:
! kaggle competitions download -c gan-getting-started

gan-getting-started.zip: Skipping, found more recently modified local copy (use --force to force download)


In [35]:
#! unzip gan-getting-started

Archive:  gan-getting-started.zip
replace monet_jpg/000c1e3bff.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [36]:
! pip install wandb weave

In [37]:
! wandb login

wandb: Currently logged in as: llodi22 (llodi22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [38]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, utils
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import wandb

# Initialize wandb
wandb.init(
    project="Monet_Paintings_Lodia",
    config={
        "image_size": 64,
        "latent_dim": 128,
        "batch_size": 64,
        "lr_G": 0.0002,
        "lr_D": 0.0001,
        "beta1": 0.5,
        "beta2": 0.999,
        "loss_type": "LSGAN_improved"
    }
)

config = wandb.config
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on:", device)


Running on: cuda


In [39]:
#!unzip -qq gan-getting-started.zip -d gan_data

replace gan_data/monet_jpg/000c1e3bff.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [40]:
from glob import glob
from PIL import Image

class MonetDataset(torch.utils.data.Dataset):
    """
    Loads all JPG files inside a folder (no class subfolders required).
    """
    def __init__(self, root, transform=None):
        self.files = sorted(glob(root + "/*.jpg"))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img = Image.open(self.files[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0   # return dummy label for compatibility


In [41]:
data_path = "/content/gan_data/monet_jpg"

transform = transforms.Compose([
    transforms.Resize((config.image_size, config.image_size)),
    transforms.RandomHorizontalFlip(p=0.5),     # prevents D from overfitting
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))        # maps RGB to [-1, 1]
])

dataset = MonetDataset(data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=config.batch_size, shuffle=True)

print("Dataset size:", len(dataset))


Dataset size: 300


In [42]:
class Generator(nn.Module):
    """
    A simple DCGAN-style generator.
    Takes a latent vector z and upsamples it step by step to produce a 3×64×64 image.

    Structure:
    - z → (fully projected) feature map → ConvTranspose2d blocks → RGB image
    - Tanh ensures output is in [-1, 1]
    """

    def __init__(self, latent_dim=100, feature_maps=64):
        super().__init__()
        self.gen = nn.Sequential(
            # Input: latent_dim × 1 × 1 → feature_maps*8 × 4 × 4
            nn.ConvTranspose2d(latent_dim, feature_maps * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(feature_maps * 8),
            nn.ReLU(True),

            # 4×4 → 8×8
            nn.ConvTranspose2d(feature_maps * 8, feature_maps * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 4),
            nn.ReLU(True),

            # 8×8 → 16×16
            nn.ConvTranspose2d(feature_maps * 4, feature_maps * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps * 2),
            nn.ReLU(True),

            # 16×16 → 32×32
            nn.ConvTranspose2d(feature_maps * 2, feature_maps, 4, 2, 1, bias=False),
            nn.BatchNorm2d(feature_maps),
            nn.ReLU(True),

            # 32×32 → 64×64 → output image
            nn.ConvTranspose2d(feature_maps, 3, 4, 2, 1, bias=False),
            nn.Tanh()  # output range must be [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)


In [43]:
class Discriminator(nn.Module):
    def __init__(self, feature_maps=64):
        super().__init__()
        self.disc = nn.Sequential(
            # 64x64 → 32x32
            nn.utils.spectral_norm(
                nn.Conv2d(3, feature_maps, 4, 2, 1, bias=False)
            ),
            nn.LeakyReLU(0.2, inplace=True),

            # 32x32 → 16x16
            nn.utils.spectral_norm(
                nn.Conv2d(feature_maps, feature_maps * 2, 4, 2, 1, bias=False)
            ),
            nn.LeakyReLU(0.2, inplace=True),

            # 16x16 → 8x8
            nn.utils.spectral_norm(
                nn.Conv2d(feature_maps * 2, feature_maps * 4, 4, 2, 1, bias=False)
            ),
            nn.LeakyReLU(0.2, inplace=True),

            # 8x8 → 4x4
            nn.utils.spectral_norm(
                nn.Conv2d(feature_maps * 4, feature_maps * 8, 4, 2, 1, bias=False)
            ),
            nn.LeakyReLU(0.2, inplace=True),

            # final output: 1×1×1 score
            nn.utils.spectral_norm(
                nn.Conv2d(feature_maps * 8, 1, 4, 1, 0, bias=False)
            )
        )

    def forward(self, x):
        return self.disc(x)


In [44]:
G = Generator(config.latent_dim).to(device)
D = Discriminator().to(device)

optimizer_G = optim.Adam(G.parameters(), lr=config.lr_G, betas=(config.beta1, config.beta2))
optimizer_D = optim.Adam(D.parameters(), lr=config.lr_D, betas=(config.beta1, config.beta2))


# LSGAN uses MSE loss
criterion = nn.MSELoss()

# fixed noise for generating sample images
fixed_noise = torch.randn(25, config.latent_dim, 1, 1, device=device)


In [45]:
epochs = 50  # LSGAN needs time

for epoch in range(epochs):
    for i, (real, _) in enumerate(dataloader):

        real = real.to(device)
        bsz = real.size(0)

        # -----------------------------
        #  Generate fake images
        # -----------------------------
        noise = torch.randn(bsz, config.latent_dim, 1, 1, device=device)
        fake = G(noise)

        # ==========================================================
        # 1. Train Discriminator (LSGAN with improvements)
        # ==========================================================
        optimizer_D.zero_grad()

        # Label smoothing for real images → improved stability
        target_real = torch.full((bsz, 1, 1, 1), 0.9, device=device)
        target_fake = torch.zeros(bsz, 1, 1, 1, device=device)

        # Optional: clamp outputs to prevent exploding gradients
        D_real = torch.clamp(D(real), -1, 1)
        D_fake = torch.clamp(D(fake.detach()), -1, 1)

        loss_real = criterion(D_real, target_real)
        loss_fake = criterion(D_fake, target_fake)
        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizer_D.step()

        # ==========================================================
        # 2. Train Generator (LSGAN)
        # ==========================================================
        optimizer_G.zero_grad()

        D_fake_for_G = D(fake)
        target_gen = torch.ones(bsz, 1, 1, 1, device=device)  # wants D(fake) → 1

        loss_G = criterion(D_fake_for_G, target_gen)
        loss_G.backward()
        optimizer_G.step()

        # wandb logging
        wandb.log({
            "loss_D": loss_D.item(),
            "loss_G": loss_G.item(),
        })

    # ==========================================================
    # Generate sample images at end of epoch
    # ==========================================================
    with torch.no_grad():
        samples = G(fixed_noise).cpu()
        grid = utils.make_grid(samples, nrow=5, normalize=True, value_range=(-1, 1))
        wandb.log({"generated_samples": wandb.Image(grid)})

    print(f"Epoch [{epoch+1}/{epochs}]  Loss_D: {loss_D.item():.4f}  Loss_G: {loss_G.item():.4f}")



Epoch [1/50]  Loss_D: 0.4974  Loss_G: 0.7981
Epoch [2/50]  Loss_D: 0.2347  Loss_G: 1.3674
Epoch [3/50]  Loss_D: 0.1089  Loss_G: 1.9406
Epoch [4/50]  Loss_D: 0.1239  Loss_G: 0.9257
Epoch [5/50]  Loss_D: 0.0783  Loss_G: 1.4711
Epoch [6/50]  Loss_D: 0.1392  Loss_G: 1.1218
Epoch [7/50]  Loss_D: 0.0687  Loss_G: 1.3763
Epoch [8/50]  Loss_D: 0.1438  Loss_G: 0.8831
Epoch [9/50]  Loss_D: 0.1898  Loss_G: 1.0358
Epoch [10/50]  Loss_D: 0.3892  Loss_G: 1.2249
Epoch [11/50]  Loss_D: 0.2850  Loss_G: 0.9422
Epoch [12/50]  Loss_D: 0.3541  Loss_G: 1.1959
Epoch [13/50]  Loss_D: 0.6606  Loss_G: 0.8672
Epoch [14/50]  Loss_D: 0.4957  Loss_G: 0.6800
Epoch [15/50]  Loss_D: 0.3478  Loss_G: 0.7968
Epoch [16/50]  Loss_D: 0.3984  Loss_G: 0.6091
Epoch [17/50]  Loss_D: 0.3208  Loss_G: 0.7725
Epoch [18/50]  Loss_D: 0.3560  Loss_G: 0.5543
Epoch [19/50]  Loss_D: 0.3922  Loss_G: 0.7139
Epoch [20/50]  Loss_D: 0.4945  Loss_G: 0.6833
Epoch [21/50]  Loss_D: 0.5193  Loss_G: 0.8326
Epoch [22/50]  Loss_D: 0.5066  Loss_G: 0.71

In [46]:
wandb.finish()

loss_D,▂▁▂▂▂▂▅▄▄▄▄▅▇▅▄▄▅▇▇███▇▇▄▇▇▇▇▇▇▇▇▇▇▇▇█▇▇
loss_G,▅▇▆▇▆▄▆▅▃▅█▇▄▂▃▂▂▂▂▂▂▁▁▂▂▂▁▁▁▁▁▁▁▁▁▂▁▁▁▁
loss_D,0.45745
loss_G,0.55941
